In [1]:
import pandas as pd
import requests
import json
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore') 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [3]:
bus = pd.read_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/버스정류장/around_bus_info_2km_08161152.pickle')
bus_cp = bus[['정류장번호','정류장명','위도','경도']]
bus_cp.reset_index(drop=True, inplace=True)

bus_cp['location'] = pd.Series(zip(bus_cp['위도'],bus_cp['경도']))
bus_cp.head()

,정류장번호,정류장명,위도,경도,location
0,ASB285000077,성환8리,36.921468,127.122774,"(36.92146807, 127.12277401)"
1,ASB285000078,송덕,36.923783,127.118976,"(36.92378302, 127.11897568)"
2,ASB285000079,국립축산과학원,36.925931,127.111136,"(36.92593139, 127.11113568)"
3,ASB285000094,성환8리,36.921479,127.122505,"(36.9214794, 127.1225049)"
4,ASB285000095,성환터미널,36.917221,127.132732,"(36.91722136, 127.13273235)"


In [7]:
water = pd.read_csv('/Users/iseunghyeon/Desktop/프로젝트/data/하천정보/전국하천위도경도_xy.csv')
water['location'] = pd.Series(zip(water['y'],water['x']))
water.head(2)
len(water)
# water['A1'].value_counts()

,A0,A1,A2,A3,x,y,location
0,710,64800004888020200241UJB4000001030,하천/용도구역,2023/03/16,127.908859,35.848407,"(35.8484069199, 127.9088593132)"
1,711,64800004888020200241UJB4000001031,하천/용도구역,2023/03/16,127.908469,35.841851,"(35.841851007, 127.9084686265)"


9796

In [8]:
# x : 경도, y : 위도
# data['distance'] = data.apply(lambda x: geopy.distance.distance(x['merch_coord'], x['cust_coord']).km, axis=1)
def calculate_distance(bus_cp, company_house):
    import geopy.distance

    data_list = []

    for idx in tqdm(range(len(bus_cp))):
        for jdx in range(len(company_house)):
            
            distance = geopy.distance.distance(bus_cp.at[idx,'location'],company_house.at[jdx,'location']).km

            data = {'정류장번호':bus_cp.at[idx,'정류장번호'],'정류장명':bus_cp.at[idx,'정류장명'],'location':bus_cp.at[idx,'location'],
            'A1':company_house.at[jdx,'A1'],'하천좌표':company_house.at[jdx,'location'],'distance':distance}
            
            data_list.append(data)

    data_df = pd.DataFrame(data_list)
    return data_df
    
# company_house.reset_index(drop=True,inplace=True)
bus_target_distance = calculate_distance(bus_cp,water)
bus_target_distance

100%|██████████| 4324/4324 [1:46:18<00:00,  1.48s/it]


,정류장번호,정류장명,location,A1,하천좌표,distance
0,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001030,"(35.8484069199, 127.9088593132)",138.392944
1,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001031,"(35.841851007, 127.9084686265)",139.003014
2,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001032,"(35.8212457233, 127.9270214296)",141.822724
3,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001033,"(35.8008020818, 127.9191018383)",143.429248
4,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64800004888020200241UJB4000001034,"(35.7985528608, 127.9189751745)",143.640528
...,...,...,...,...,...,...
42357899,GGB500000184,둔포,"(36.9302833, 127.0387333)",65100005011020220363UJB1000300001,"(33.4587157694, 126.6011334201)",387.205289
42357900,GGB500000184,둔포,"(36.9302833, 127.0387333)",65100005011020220363UJB1000400001,"(33.4788453515, 126.5674488021)",385.313401
42357901,GGB500000184,둔포,"(36.9302833, 127.0387333)",65100005011020220363UJB1000500001,"(33.4656083507, 126.562296228)",386.825144
42357902,GGB500000184,둔포,"(36.9302833, 127.0387333)",65100005011020220363UJB1000600001,"(33.4520883482, 126.5565717735)",388.374211


In [9]:
# bus_target_distance.to_pickle('/Users/iseunghyeon/Desktop/프로젝트/data/하천정보/버스정류장과하천과의거리.pickle')

In [11]:
bus_target_distance[bus_target_distance['distance']<=2]

,정류장번호,정류장명,location,A1,하천좌표,distance
7736,ASB285000077,성환8리,"(36.92146807, 127.12277401)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.398270
17532,ASB285000078,송덕,"(36.92378302, 127.11897568)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.813301
27328,ASB285000079,국립축산과학원,"(36.92593139, 127.11113568)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",1.545798
37124,ASB285000094,성환8리,"(36.9214794, 127.1225049)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.421355
46920,ASB285000095,성환터미널,"(36.91722136, 127.13273235)",64400004413020210447UJB1000000007,"(36.9202740809, 127.1269891067)",0.613733
...,...,...,...,...,...,...
42356007,GGB500000184,둔포,"(36.9302833, 127.0387333)",64400004420020140281UJB1000003001,"(36.9285764656, 127.032786977)",0.562625
42356008,GGB500000184,둔포,"(36.9302833, 127.0387333)",64400004420020140281UJB1000003002,"(36.9298423943, 127.0346123897)",0.370391
42356009,GGB500000184,둔포,"(36.9302833, 127.0387333)",64400004420020140281UJB1000003003,"(36.9325316743, 127.0414696927)",0.348843
42356010,GGB500000184,둔포,"(36.9302833, 127.0387333)",64400004420020140281UJB1000003004,"(36.933895686, 127.0489257222)",0.992607
